In [ ]:

#include <IRremote.h>
#include <Stepper.h>
#include <SPI.h>
#include <MFRC522.h>

#define RECV_PIN 5
#define RST_PIN 9
#define SS_PIN 10
#define TRIG_PIN 7
#define ECHO_PIN 6

IRrecv irrecv(RECV_PIN);
decode_results results;

int dir;
const int stepsPerRevolution = 200;
Stepper myStepper(stepsPerRevolution, 8, 9, 10, 11); // Arduino pins to H-bridge inputs

MFRC522 rfid(SS_PIN, RST_PIN); // Create MFRC522 instance

void setup() {
    myStepper.setSpeed(60);
    Serial.begin(9600);
    irrecv.enableIRIn(); // Start the receiver

    // Initialize RFID
    SPI.begin();
    rfid.PCD_Init();
    Serial.println("RFID reader initialized.");

    // Initialize Ultrasonic sensor
    pinMode(TRIG_PIN, OUTPUT);
    pinMode(ECHO_PIN, INPUT);
}

void loop() {
    // Handle IR remote signals
    if (irrecv.decode(&results)) {
        Serial.println(results.value);
        irrecv.resume(); // Receive the next value
        if (results.value == 1320358637) {
            myStepper.step(+1);
            dir = 1;
        } else if (results.value == 4294967295 && dir == 1) {
            myStepper.step(+1);
        } else if (results.value == 1320368327) {
            myStepper.step(-1);
            dir = -1;
        } else if (results.value == 4294967295 && dir == -1) {
            myStepper.step(-1);
        } else if (results.value == 1320392807) {
            myStepper.step(+100);
            dir = 100;
        } else if (results.value == 4294967295 && dir == 100) {
            myStepper.step(+100);
        } else if (results.value == 1320360167) {
            myStepper.step(-100);
            dir = -100;
        } else if (results.value == 4294967295 && dir == -100) {
            myStepper.step(-100);
        } else if (results.value == 1320401477) {
            myStepper.step(+1000);
            dir = 1000;
        } else if (results.value == 4294967295 && dir == 1000) {
            myStepper.step(+1000);
        } else if (results.value == 1320417287) {
            myStepper.step(-1000);
            dir = -1000;
        } else if (results.value == 4294967259 && dir == -1000) {
            myStepper.step(-1000);
        } else dir = 0;
    }

    // Handle RFID reading
    if (rfid.PICC_IsNewCardPresent() && rfid.PICC_ReadCardSerial()) {
        Serial.print("Card UID: ");
        for (byte i = 0; i < rfid.uid.size; i++) {
            Serial.print(rfid.uid.uidByte[i] < 0x10 ? " 0" : " ");
            Serial.print(rfid.uid.uidByte[i], HEX);
        }
        Serial.println();
        rfid.PICC_HaltA();
    }

    // Handle Ultrasonic sensor
    digitalWrite(TRIG_PIN, LOW);
    delayMicroseconds(2);
    digitalWrite(TRIG_PIN, HIGH);
    delayMicroseconds(10);
    digitalWrite(TRIG_PIN, LOW);
    long duration = pulseIn(ECHO_PIN, HIGH);
    long distance = duration * 0.034 / 2;

    Serial.print("Distance: ");
    Serial.print(distance);
    Serial.println(" cm");

    delay(5);
}
